In [1]:
import numpy as np
import xarray as xr
import pandas as pd
import geopandas as gpd
import h5py
import os,json
from pathlib import Path
from pprint import pprint
import dask.dataframe as dd
import geopandas as gpd
from shapely.geometry import Point
import rasterio
import numpy as np
from pathlib import Path

In [2]:
# import geopandas as gpd
# gpd.show_versions()  # 看 fiona/pyogrio/gdal 是否都有版本号

In [3]:
# import sys, geopandas as gpd, shapely
# print("PYTHON:", sys.executable)
# print("GEOPANDAS:", gpd.__file__)

### 提取参数
- 掩膜切片
- 参数提取
- 存为dataframe格式

### Parameter Extraction
- Mask slicing
- Parameter extraction
- Saved in DataFrame format

In [14]:
def read_atl08(fname, bbox=None, poly=None, strong_only=False):
    group = ['/gt1l','/gt1r','/gt2l','/gt2r','/gt3l','/gt3r']
    all_beams = []
    

    with h5py.File(fname, 'r') as fi:
        for g in group:
            # only leave strong beam
            if strong_only and g not in ['/gt2l', '/gt2r']:
                continue
            ls = f"{g}/land_segments"
            if ls not in fi:
                continue
            grp = fi[ls]

            lat = grp["latitude"][:]
            lon = grp["longitude"][:]

            if bbox is not None:
                lonmin, lonmax, latmin, latmax = bbox
                m = (lon >= lonmin) & (lon <= lonmax) & (lat >= latmin) & (lat <= latmax)
                if not np.any(m):
                    continue
                # only reserve the true of m
                lat, lon = lat[m], lon[m]

            def take(path, is2d=False):
                arr = grp[path][:]
                return arr[m, :] if (bbox is not None and is2d) else (arr[m] if bbox is not None else arr)

            dem_h         = take("dem_h")
            h_te_interp   = take("terrain/h_te_interp")
            h_te_mean     = take("terrain/h_te_mean")
            h_te_best_fit = take("terrain/h_te_best_fit")
            terrain_flg   = take("terrain_flg")
            urban_flag    = take("urban_flag")
            cloud_flag_atm= take("cloud_flag_atm")
            terrain_slope = take("terrain/terrain_slope")
            n_te_photons  = take("terrain/n_te_photons")

            df = pd.DataFrame({
                "beam": g[1:], "lat": lat, "lon": lon,
                "dem_h": dem_h, "h_te_interp": h_te_interp, "h_te_mean": h_te_mean,
                "h_te_best_fit": h_te_best_fit, "terrain_flg": terrain_flg,
                "urban_flag": urban_flag, "cloud_flag_atm": cloud_flag_atm,
                "terrain_slope": terrain_slope, "n_te_photons": n_te_photons
            })
            all_beams.append(df)
       
    if poly is not None and len(all_beams) > 0:
        df_all = pd.concat(all_beams, ignore_index=True)
        # 将 lon/lat 转成点要素
        pts = gpd.GeoSeries([Point(xy) for xy in zip(df_all["lon"], df_all["lat"])], crs="EPSG:4326")
        poly_geom = poly.union_all()
        # within：点在多边形内部（严格意义）；如果想包含边界可用 .covers()   
        in_poly = pts.within(poly_geom)
        # 用多边形掩膜过滤 重置序号
        df_all = df_all.loc[in_poly].reset_index(drop=True)
        return df_all


    return pd.concat(all_beams, ignore_index=True) if all_beams else pd.DataFrame()

### 打印树结构
- 看所有变量位置
### tree structure

In [3]:
fname = r'/Users/a1-6/Library/Mobile Documents/com~apple~CloudDocs/University/Polimi/研二上/GP/GP_program/GP_Project/tests/data/ATL08_20220102113835_01711406_007_01.h5'

with h5py.File(fname, 'r') as f:
    def print_tree(name, obj):
        print(name)
    f.visititems(print_tree)

METADATA
METADATA/AcquisitionInformation
METADATA/AcquisitionInformation/lidar
METADATA/AcquisitionInformation/lidarDocument
METADATA/AcquisitionInformation/platform
METADATA/AcquisitionInformation/platformDocument
METADATA/DatasetIdentification
METADATA/Extent
METADATA/Lineage
METADATA/Lineage/ANC06-01
METADATA/Lineage/ANC06-02
METADATA/Lineage/ANC06-03
METADATA/Lineage/ANC14
METADATA/Lineage/ANC18
METADATA/Lineage/ANC19
METADATA/Lineage/ANC25-08
METADATA/Lineage/ANC26-08
METADATA/Lineage/ANC28
METADATA/Lineage/ANC33
METADATA/Lineage/ANC36-08
METADATA/Lineage/ANC38-08
METADATA/Lineage/ANC46
METADATA/Lineage/ANC47
METADATA/Lineage/ATL03
METADATA/Lineage/ATL09
METADATA/Lineage/Control
METADATA/ProcessStep
METADATA/ProcessStep/Browse
METADATA/ProcessStep/Metadata
METADATA/ProcessStep/PGE
METADATA/ProcessStep/QA
METADATA/ProductSpecificationDocument
METADATA/QADatasetIdentification
METADATA/SeriesIdentification
METADATA/iso_19139_dataset_xml
METADATA/iso_19139_series_xml
ancillary_data
an

### 测试read_at108函数

### test for the function

In [15]:
fname = r"/Users/a1-6/Self_program/underground_water/ATL08_007-20251002_204153/ATL08_20220102113835_01711406_007_01.h5"

df = read_atl08(fname, poly=poly, strong_only=True)

print("数据总行数:", len(df))
print("列名:", df.columns.tolist())
print("前几行:")
print(df.head())

# 简单统计一下关键字段
print("\nlat 范围:", df["lat"].min(), "→", df["lat"].max())
print("lon 范围:", df["lon"].min(), "→", df["lon"].max())
print("高程范围:", df["h_te_best_fit"].min(), "→", df["h_te_best_fit"].max())

# 按束分组统计数量
print("\n每束点数统计:")
print(df["beam"].value_counts())

数据总行数: 43
列名: ['beam', 'lat', 'lon', 'dem_h', 'h_te_interp', 'h_te_mean', 'h_te_best_fit', 'terrain_flg', 'urban_flag', 'cloud_flag_atm', 'terrain_slope', 'n_te_photons']
前几行:
   beam        lat         lon     dem_h  h_te_interp  h_te_mean  \
0  gt2l  29.210592  115.817505  8.638745     7.509477   7.455386   
1  gt2l  29.209692  115.817413  5.274000     7.480859   7.484968   
2  gt2l  29.208792  115.817307  3.451258     7.494159   7.489066   
3  gt2l  29.206091  115.817001  3.622857     7.494613   7.494867   
4  gt2l  29.205189  115.816902  3.618638     7.496910   7.496326   

   h_te_best_fit  terrain_flg  urban_flag  cloud_flag_atm  terrain_slope  \
0       7.545080            1           0               2       0.001786   
1       7.486938            1           0               2       0.000135   
2       7.487191            1           0               2      -0.000094   
3       7.491868            1           0               2       0.000098   
4       7.492458            1      

### 生成位掩码 bitmask
- `masks` 字典：每个点单独条件是否通过（方便统计单个条件的通过率）。
- `bitmask` 数组：每个点一个整数，记录它通过了哪些条件。
    - 例如 bitmask = `0b010101101` → 表示通过了 a, c, e, g, i。
- masks dictionary: indicates whether each individual condition is passed for each point (useful for computing per-condition pass rates).
- bitmask array: one integer per point, encoding which conditions it passed.


In [16]:
def build_masks(df, sigma_a_b=25.0, thr_cloud=3, thr_nphot=20, thr_slope=0.01):
    """返回 a..i 的布尔列，以及 bitmask（int32）"""
    # a: |dem_h - h_te_interp| <= 3σ（σ取25 m）
    a = (np.abs(df.dem_h - df.h_te_interp) <= 3 * sigma_a_b)

    # b: |h_te_mean - h_te_interp| <= 3σ（σ取25 m）
    b = (np.abs(df.h_te_mean - df.h_te_interp) <= 3 * sigma_a_b)

    # c: terrain_flg == 0
    c = (df.terrain_flg == 0)

    # d: urban_flag == 0
    d = (df.urban_flag == 0)

    # e: cloud_flag_atm < 3
    e = (df.cloud_flag_atm < thr_cloud)

    # f: terrain_slope < 0.01
    f = (df.terrain_slope < thr_slope)

    # g: n_te_photons > 20
    g = (df.n_te_photons > thr_nphot)

    # h: |dem_h - h_te_interp| <= 3
    h = (np.abs(df.dem_h - df.h_te_interp) <= 3.0)

    # i: |dem_h - h_te_best_fit| <= 3
    i = (np.abs(df.dem_h - df.h_te_best_fit) <= 3.0)

    masks = {'a':a,'b':b,'c':c,'d':d,'e':e,'f':f,'g':g,'h':h,'i':i}

    bitmask = np.zeros(len(df), dtype=np.int32)
    for bit,key in enumerate(['a','b','c','d','e','f','g','h','i']):
        bitmask |= (masks[key].values.astype(np.int32) << bit)

    return masks, bitmask

## 和水体进行叠加分析
## overlayer anaysis with water body

In [17]:
import re

def extract_yyyymm_from_filename(fname):
    """
    从 ATL08 文件名中提取 yyyyMM，例如:
    'ATL08_20220115225748_03771402_007_01.h5' → '202201'
    """
    # 其中 前 6 位保存为结果，后 2 位只是用于匹配
    m = re.search(r"ATL08_(\d{6})\d{2}", fname)
    if m:
        return m.group(1)  # yyyyMM
    else:
        raise ValueError(f"无法从文件名中提取年月: {fname}")
        

In [18]:
def drop_water_points_with_tiff(df, tiff_path, lon_col="lon", lat_col="lat"):
    if df.empty:
        return df

    tiff_path = Path(tiff_path)
    if not tiff_path.exists():
        print(f"找不到水体栅格 {tiff_path} — 跳过去水体步骤")
        return df

    with rasterio.open(tiff_path) as src:
        coords = list(zip(df[lon_col].values, df[lat_col].values))
        samples = list(src.sample(coords))
        vals = np.array([s[0] if s is not None else np.nan for s in samples])

    keep = (vals == 2)          # 2 代表陆地  
    df_filtered = df.loc[keep].copy()

    print(f"  - {tiff_path.name}: 原 {len(df)} 点 → 剩 {len(df_filtered)} 个陆地点（已去掉水体）")
    return df_filtered
    

In [19]:
def drop_water_points_monthly(df, fname, tiff_folder, lon_col="lon", lat_col="lat"):
    """
    根据点数据的文件名自动选择对应月份的水体 TIFF 并过滤水体点。

    参数:
    df           : read_atl08 返回的点 DataFrame
    fname        : 当前 h5 文件的路径，用于提取日期
    tiff_folder  : 存放 12 个月水体 TIFF 的文件夹
    """

    # 1. 从文件名提取 YYYYMM
    yyyymm = extract_yyyymm_from_filename(fname)
    # 拼成一个合法路径
    tiff_path = os.path.join(tiff_folder, f"{yyyymm}.tif")

    if not os.path.exists(tiff_path):
        print(f"找不到对应月份的水体文件：{tiff_path}，跳过水体筛除")
        return df

    # 2. 使用上面写好的 drop_water_points_with_tiff
    df_filtered = drop_water_points_with_tiff(df, tiff_path, lon_col=lon_col, lat_col=lat_col)
    print(f"✔ {yyyymm} 去水体后剩余点数：{len(df_filtered)}")
    return df_filtered
    

### 测试
- 输出各个约束的通过率

In [20]:
# fname = r'/Users/a1-6/Self_program/underground_water/ATL08_007-20251002_204153/ATL08_20220102113835_01711406_007_01.h5'
# bbox = (115.5, 116.8, 28.2, 29.9) 
# df = read_atl08(fname, bbox=bbox)
# masks, bitmask = build_masks(df)

# for k,v in masks.items():
#     print(f"{k}: {v.sum()} / {len(v)}  ({v.mean():.2%})")

# need = ['a','b','c','d','e']
# order = ['a','b','c','d','e','f','g','h','i']
# need_bits = sum(1 << order.index(k) for k in need)
# ok = (bitmask & need_bits) == need_bits

# print(f"法则4 通过: {ok.sum()} / {len(ok)}")
# df_rule4 = df[ok]  
# df_rule4.to_csv("controlpoints_rule4.csv", index=False)

In [21]:
# 2019-2020 11-1月
# files = [
#     r'/Users/a1-6/Self_program/underground_water/ICESat-2_Data/2019-2020_11 _12_1/ATL08_20191102012423_05520506_007_01.h5',
#     r'/Users/a1-6/Self_program/underground_water/ICESat-2_Data/2019-2020_11 _12_1/ATL08_20191106011604_06130506_007_01.h5',
#     r'/Users/a1-6/Self_program/underground_water/ICESat-2_Data/2019-2020_11 _12_1/ATL08_20191110010744_06740506_007_01.h5',
#     r'/Users/a1-6/Self_program/underground_water/ICESat-2_Data/2019-2020_11 _12_1/ATL08_20191115124337_07580502_007_01.h5',
#     r'/Users/a1-6/Self_program/underground_water/ICESat-2_Data/2019-2020_11 _12_1/ATL08_20191119123516_08190502_007_01.h5',
#     r'/Users/a1-6/Self_program/underground_water/ICESat-2_Data/2019-2020_11 _12_1/ATL08_20191123122656_08800502_007_01.h5',
#     r'/Users/a1-6/Self_program/underground_water/ICESat-2_Data/2019-2020_11 _12_1/ATL08_20191204235205_10550506_007_01.h5',
#     r'/Users/a1-6/Self_program/underground_water/ICESat-2_Data/2019-2020_11 _12_1/ATL08_20191208234345_11160506_007_01.h5',
#     r'/Users/a1-6/Self_program/underground_water/ICESat-2_Data/2019-2020_11 _12_1/ATL08_20200106221946_01710606_007_01.h5',
#     r'/Users/a1-6/Self_program/underground_water/ICESat-2_Data/2019-2020_11 _12_1/ATL08_20200116094716_03160602_007_01.h5',
#     r'/Users/a1-6/Self_program/underground_water/ICESat-2_Data/2019-2020_11 _12_1/ATL08_20200120093856_03770602_007_01.h5',
#     r'/Users/a1-6/Self_program/underground_water/ICESat-2_Data/2019-2020_11 _12_1/ATL08_20200131210410_05520606_007_01.h5'
       
# ]

#2020-2021 11-1月
# files = [
#     r'/Users/a1-6/Self_program/underground_water/ICESat-2_Data/2020-2021_11_12_1/ATL08_20201103075511_06130906_007_01.h5',
#     r'/Users/a1-6/Self_program/underground_water/ICESat-2_Data/2020-2021_11_12_1/ATL08_20201107074651_06740906_007_01.h5',
#     r'/Users/a1-6/Self_program/underground_water/ICESat-2_Data/2020-2021_11_12_1/ATL08_20201112192242_07580902_007_01.h5',
#     r'/Users/a1-6/Self_program/underground_water/ICESat-2_Data/2020-2021_11_12_1/ATL08_20201116191424_08190902_007_01.h5',
#     r'/Users/a1-6/Self_program/underground_water/ICESat-2_Data/2020-2021_11_12_1/ATL08_20201120190605_08800902_007_01.h5',
#     r'/Users/a1-6/Self_program/underground_water/ICESat-2_Data/2020-2021_11_12_1/ATL08_20201206062256_11160906_007_01.h5',
#     r'/Users/a1-6/Self_program/underground_water/ICESat-2_Data/2020-2021_11_12_1/ATL08_20201215175029_12610902_007_01.h5',
#     r'/Users/a1-6/Self_program/underground_water/ICESat-2_Data/2020-2021_11_12_1/ATL08_20201219174209_13220902_007_01.h5',
#     r'/Users/a1-6/Self_program/underground_water/ICESat-2_Data/2020-2021_11_12_1/ATL08_20201231050721_01101006_007_01.h5',
#     r'/Users/a1-6/Self_program/underground_water/ICESat-2_Data/2020-2021_11_12_1/ATL08_20210104045901_01711006_007_01.h5',
#     r'/Users/a1-6/Self_program/underground_water/ICESat-2_Data/2020-2021_11_12_1/ATL08_20210113162633_03161002_007_01.h5',
#     r'/Users/a1-6/Self_program/underground_water/ICESat-2_Data/2020-2021_11_12_1/ATL08_20210117161809_03771002_007_01.h5',
#     r'/Users/a1-6/Self_program/underground_water/ICESat-2_Data/2020-2021_11_12_1/ATL08_20210129034327_05521006_007_01.h5'
# ]



# 2021-2022 11-1月
# files = [
#     r'/Users/a1-6/Self_program/underground_water/ICESat-2_Data/2021-2022_11_12_1/ATL08_20211101143447_06131306_007_01.h5',
#     r'/Users/a1-6/Self_program/underground_water/ICESat-2_Data/2021-2022_11_12_1/ATL08_20211105142631_06741306_007_01.h5',
#     r'/Users/a1-6/Self_program/underground_water/ICESat-2_Data/2021-2022_11_12_1/ATL08_20211111020221_07581302_007_01.h5',
#     r'/Users/a1-6/Self_program/underground_water/ICESat-2_Data/2021-2022_11_12_1/ATL08_20211115015401_08191302_007_01.h5',
#     r'/Users/a1-6/Self_program/underground_water/ICESat-2_Data/2021-2022_11_12_1/ATL08_20211119014545_08801302_007_01.h5',
#     r'/Users/a1-6/Self_program/underground_water/ICESat-2_Data/2021-2022_11_12_1/ATL08_20211126131911_09941306_007_01.h5',
#     r'/Users/a1-6/Self_program/underground_water/ICESat-2_Data/2021-2022_11_12_1/ATL08_20211130131051_10551306_007_01.h5',
#     r'/Users/a1-6/Self_program/underground_water/ICESat-2_Data/2021-2022_11_12_1/ATL08_20211204130231_11161306_007_01.h5',
#     r'/Users/a1-6/Self_program/underground_water/ICESat-2_Data/2021-2022_11_12_1/ATL08_20211214003005_12611302_007_01.h5',
#     r'/Users/a1-6/Self_program/underground_water/ICESat-2_Data/2021-2022_11_12_1/ATL08_20211218002145_13221302_007_01.h5',
#     r'/Users/a1-6/Self_program/underground_water/ICESat-2_Data/2021-2022_11_12_1/ATL08_20211229114654_01101406_007_01.h5',
#     r'/Users/a1-6/Self_program/underground_water/ICESat-2_Data/2021-2022_11_12_1/ATL08_20220102113835_01711406_007_01.h5',
#     r'/Users/a1-6/Self_program/underground_water/ICESat-2_Data/2021-2022_11_12_1/ATL08_20220111230607_03161402_007_01.h5',
#     r'/Users/a1-6/Self_program/underground_water/ICESat-2_Data/2021-2022_11_12_1/ATL08_20220115225748_03771402_007_01.h5',
#     r'/Users/a1-6/Self_program/underground_water/ICESat-2_Data/2021-2022_11_12_1/ATL08_20220119224927_04381402_007_01.h5'
# ]
#2022-2023 11-1月
# files = [
#     r'/Users/a1-6/Self_program/underground_water/ICESat-2_Data/2022-2023_11_12_1/ATL08_20221103210552_06741706_007_01.h5',
#     r'/Users/a1-6/Self_program/underground_water/ICESat-2_Data/2022-2023_11_12_1/ATL08_20221109084141_07581702_007_01.h5',
#     r'/Users/a1-6/Self_program/underground_water/ICESat-2_Data/2022-2023_11_12_1/ATL08_20221113083320_08191702_007_01.h5',
#     r'/Users/a1-6/Self_program/underground_water/ICESat-2_Data/2022-2023_11_12_1/ATL08_20221117082503_08801702_007_01.h5',
#     r'/Users/a1-6/Self_program/underground_water/ICESat-2_Data/2022-2023_11_12_1/ATL08_20221124195834_09941706_007_01.h5',
#     r'/Users/a1-6/Self_program/underground_water/ICESat-2_Data/2022-2023_11_12_1/ATL08_20221128195015_10551706_007_01.h5',
#     r'/Users/a1-6/Self_program/underground_water/ICESat-2_Data/2022-2023_11_12_1/ATL08_20221202194153_11161706_007_01.h5',
#     r'/Users/a1-6/Self_program/underground_water/ICESat-2_Data/2022-2023_11_12_1/ATL08_20221212070923_12611702_007_01.h5',
#     r'/Users/a1-6/Self_program/underground_water/ICESat-2_Data/2022-2023_11_12_1/ATL08_20221216070101_13221702_007_01.h5',
#     r'/Users/a1-6/Self_program/underground_water/ICESat-2_Data/2022-2023_11_12_1/ATL08_20230104180933_02321806_007_01.h5',
#     r'/Users/a1-6/Self_program/underground_water/ICESat-2_Data/2022-2023_11_12_1/ATL08_20230110054523_03161802_007_01.h5',
#     r'/Users/a1-6/Self_program/underground_water/ICESat-2_Data/2022-2023_11_12_1/ATL08_20230129165410_06131806_007_01.h5'
# ]
# 2022 12月
# files = [
#     r'/Users/a1-6/Self_program/underground_water/ICESat-2_Data/2022-2023_11_12_1/ATL08_20221202194153_11161706_007_01.h5',
#     r'/Users/a1-6/Self_program/underground_water/ICESat-2_Data/2022-2023_11_12_1/ATL08_20221212070923_12611702_007_01.h5',
#     r'/Users/a1-6/Self_program/underground_water/ICESat-2_Data/2022-2023_11_12_1/ATL08_20221216070101_13221702_007_01.h5'
# ]
# 2023-2024 11-1月
# files = [
#     r'/Users/a1-6/Self_program/underground_water/ICESat-2_Data/2023-2024_11_12_1/ATL08_20231102034411_06742106_007_01.h5',
#     r'/Users/a1-6/Self_program/underground_water/ICESat-2_Data/2023-2024_11_12_1/ATL08_20231107151954_07582102_007_01.h5',
#     r'/Users/a1-6/Self_program/underground_water/ICESat-2_Data/2023-2024_11_12_1/ATL08_20231123023701_09942106_007_01.h5',
#     r'/Users/a1-6/Self_program/underground_water/ICESat-2_Data/2023-2024_11_12_1/ATL08_20231127022828_10552106_007_01.h5',
#     r'/Users/a1-6/Self_program/underground_water/ICESat-2_Data/2023-2024_11_12_1/ATL08_20231201022008_11162106_007_01.h5',
#     r'/Users/a1-6/Self_program/underground_water/ICESat-2_Data/2023-2024_11_12_1/ATL08_20231210134751_12612102_007_01.h5',
#     r'/Users/a1-6/Self_program/underground_water/ICESat-2_Data/2023-2024_11_12_1/ATL08_20231214133923_13222102_007_01.h5',
#     r'/Users/a1-6/Self_program/underground_water/ICESat-2_Data/2023-2024_11_12_1/ATL08_20231226010431_01102206_007_01.h5',
#     r'/Users/a1-6/Self_program/underground_water/ICESat-2_Data/2023-2024_11_12_1/ATL08_20231230005610_01712206_007_01.h5',
#     r'/Users/a1-6/Self_program/underground_water/ICESat-2_Data/2023-2024_11_12_1/ATL08_20240112121516_03772202_007_01.h5',
#     r'/Users/a1-6/Self_program/underground_water/ICESat-2_Data/2023-2024_11_12_1/ATL08_20240116120651_04382202_007_01.h5',
#     r'/Users/a1-6/Self_program/underground_water/ICESat-2_Data/2023-2024_11_12_1/ATL08_20240123234022_05522206_007_01.h5',
#     r'/Users/a1-6/Self_program/underground_water/ICESat-2_Data/2023-2024_11_12_1/ATL08_20240127233201_06132206_007_01.h5',
#     r'/Users/a1-6/Self_program/underground_water/ICESat-2_Data/2023-2024_11_12_1/ATL08_20240131232342_06742206_007_01.h5',
# ]

# bbox = (115.5, 116.8, 28.2, 29.9) 

folder = Path("/Users/a1-6/Self_program/underground_water/thesis/ICESat/2022_fullYearData")
tiff_folder = "/Users/a1-6/Self_program/underground_water/thesis/poyang_waterbody_2022"

files = sorted(str(p) for p in folder.glob("ATL08_*.h5"))


#用shp文件进行掩膜
poly = gpd.read_file("/Users/a1-6/Self_program/underground_water/thesis/boundary/2022.shp")
if poly.crs is None:
    raise ValueError("Shapefile 没有坐标系（CRS），请先在 GIS 里定义或确认")
poly = poly.to_crs(4326) 

rules = {
    "Rule1": list("ab"),
    "Rule2": list("abc"),
    "Rule3": list("abcd"),
    "Rule4": list("abcde"),
    "Rule5": list("abcdef"),
    "Rule6": list("abcdefg"),
    "Rule7": list("hi"),
    "Rule8": list("abcdefghi"),
}
order = list("abcdefghi") 
bit_of = {k: i for i, k in enumerate(order)}

out_dir = Path("/Users/a1-6/Self_program/underground_water/thesis/ICESat/rulePoint_withoutWater_strongbeam")
out_dir.mkdir(parents=True, exist_ok=True)

raw_dir = Path("raw_points_out")
raw_dir.mkdir(parents=True, exist_ok=True)

all_summary = []
no_data_files = [] 

for fname in files:  
    stem = Path(fname).stem  
    df = read_atl08(fname, poly=poly, strong_only=True)
    if df is None or df.empty or ("beam" not in df.columns):
        no_data_files.append(Path(fname).name) 
        continue
    df = df.replace([np.inf, -np.inf], np.nan)

    # 按文件日期匹配对应月份水体 TIFF，去掉水体上的点
    df = drop_water_points_monthly(
        df,
        fname=fname,
        tiff_folder=tiff_folder,
        lon_col="lon",
        lat_col="lat",
    )

    raw_cols = [
        "beam", "lon", "lat",
        "h_te_best_fit", "h_te_interp", "h_te_mean",
        "dem_h", "terrain_flg", "urban_flag", "cloud_flag_atm",
        "terrain_slope", "n_te_photons"
    ]
    raw_csv_path = raw_dir / f"{stem}_raw_allpoints.csv"
    df.loc[:, raw_cols].to_csv(raw_csv_path, index=False)
    print(f"已输出未筛选点文件：{raw_csv_path.name} （共 {len(df):,} 点）")



    print(f"\n=== 处理: {stem} ===")
    print(f"掩膜后 DataFrame 总点数：{len(df):,}")
    print("各束点数统计：")
    print(df["beam"].value_counts())


    masks, bitmask = build_masks(df)
    summary = [] 
    for rule_name, need in rules.items():
        need_bits = sum(1 << bit_of[k] for k in need) 
        ok = (bitmask & need_bits) == need_bits

        cols_full = ["beam","lon","lat","h_te_best_fit","h_te_interp","h_te_mean",
                     "dem_h","terrain_flg","urban_flag","cloud_flag_atm",
                     "terrain_slope","n_te_photons"]

        df.loc[ok, cols_full].copy().to_csv(out_dir / f"{stem}_{rule_name}_full.csv", index=False)

        n_pass = int(ok.sum())
        rate = float(ok.mean()) if len(ok) else 0.0
        summary.append({
            "rule": rule_name,
            "need": "+".join(need),
            "N_pass": n_pass,
            "rate": rate
        })
    for row in summary:
        row["file"] = stem
        all_summary.append(row)
 
    pd.DataFrame(summary).to_csv(out_dir / f"{stem}_rule_summary.csv", index=False)
    print(pd.DataFrame(summary))

if no_data_files:
    print("\n以下文件在掩膜区域内没有数据：")
    for f in no_data_files:
        print("  -", f)

df_all_rules = pd.DataFrame(all_summary)

df_summary_total = (
    df_all_rules.groupby("rule", as_index=False)
    .agg({"N_pass": "sum"})
    .rename(columns={"N_pass": "Total_N_pass"})
)

total_points = 0
for fname in files:
    df_tmp = read_atl08(fname, poly=poly, strong_only=True)
    if df_tmp is None or df_tmp.empty:
        continue
    df_tmp = df_tmp.replace([np.inf, -np.inf], np.nan)
    
    # 同样按月份去水体
    df_tmp = drop_water_points_monthly(
        df_tmp,
        fname=fname,
        tiff_folder=tiff_folder,
        lon_col="lon",
        lat_col="lat",
    )

    total_points += len(df_tmp)

df_summary_total["Global_reject_rate"] = 1 - (df_summary_total["Total_N_pass"] / total_points)

df_summary_total.to_csv(out_dir / "rules_total_summary.csv", index=False)
print("\n=== 汇总 ===")
print(df_summary_total)

  - 202201.tif: 原 43 点 → 剩 1 个陆地点（已去掉水体）
✔ 202201 去水体后剩余点数：1
已输出未筛选点文件：ATL08_20220102113835_01711406_007_01_raw_allpoints.csv （共 1 点）

=== 处理: ATL08_20220102113835_01711406_007_01 ===
掩膜后 DataFrame 总点数：1
各束点数统计：
beam
gt2l    1
Name: count, dtype: int64
    rule               need  N_pass  rate  \
0  Rule1                a+b       1   1.0   
1  Rule2              a+b+c       0   0.0   
2  Rule3            a+b+c+d       0   0.0   
3  Rule4          a+b+c+d+e       0   0.0   
4  Rule5        a+b+c+d+e+f       0   0.0   
5  Rule6      a+b+c+d+e+f+g       0   0.0   
6  Rule7                h+i       0   0.0   
7  Rule8  a+b+c+d+e+f+g+h+i       0   0.0   

                                   file  
0  ATL08_20220102113835_01711406_007_01  
1  ATL08_20220102113835_01711406_007_01  
2  ATL08_20220102113835_01711406_007_01  
3  ATL08_20220102113835_01711406_007_01  
4  ATL08_20220102113835_01711406_007_01  
5  ATL08_20220102113835_01711406_007_01  
6  ATL08_20220102113835_01711406_007_01  
7  AT

### 测验——原始数据直接转换成csv


In [22]:

# 你需要的字段（都在 land_segments 下，个别在子组里）
# 左边是 CSV 列名，右边是 HDF5 路径（相对 land_segments）
ATL08_FIELDS = {
    "lat": "latitude",
    "lon": "longitude",
    "dem_h": "dem_h",  # 注意：某些版本在 dem/dem_h，如果报错见下方 fallback
    "h_te_interp": "terrain/h_te_interp",
    "h_te_mean": "terrain/h_te_mean",
    "h_te_best_fit": "terrain/h_te_best_fit",
    "terrain_flg": "terrain_flg",
    "urban_flag": "urban_flag",
    "cloud_flag_atm": "cloud_flag_atm",
    "terrain_slope": "terrain/terrain_slope",
    "n_te_photons": "terrain/n_te_photons",
}

BEAMS = ["/gt1l","/gt1r","/gt2l","/gt2r","/gt3l","/gt3r"]

def _safe_read(group, rel_path):
    """安全读取：字段不存在返回 None。"""
    try:
        return group[rel_path][:]
    except KeyError:
        return None

def atl08_h5_to_csv(h5_path, out_csv=None, per_beam=False):
    """
    将单个 ATL08 .h5 转为 CSV（无任何筛选）。
    - h5_path:  输入文件路径
    - out_csv:  输出 CSV 路径（默认与输入同名，同目录）
    - per_beam: 若为 True，按束分别输出多个 CSV；否则合并为一个 CSV
    """
    h5_path = Path(h5_path)
    if out_csv is None and not per_beam:
        out_csv = h5_path.with_suffix(".csv")

    rows_all = []
    outputs = []

    with h5py.File(h5_path, "r") as fi:
        for g in BEAMS:
            ls = f"{g}/land_segments"
            if ls not in fi:
                # 该束缺失
                continue

            grp = fi[ls]
            # 先试主要字段
            cols = {}
            for col, rel in ATL08_FIELDS.items():
                arr = _safe_read(grp, rel)
                cols[col] = arr

            # 兼容 dem_h 可能在 dem/dem_h 的情况
            if cols["dem_h"] is None:
                alt = _safe_read(grp, "dem/dem_h")
                cols["dem_h"] = alt

            # 必须有经纬度才有意义
            lat = cols["lat"]
            lon = cols["lon"]
            if lat is None or lon is None:
                continue

            n = len(lat)
            data = {
                "beam": np.full(n, g[1:]),
                "lat": lat,
                "lon": lon,
            }

            # 其他列对齐长度（不存在则全 NaN）
            for col in ATL08_FIELDS.keys():
                if col in ("lat","lon"):
                    continue
                arr = cols[col]
                if arr is None:
                    data[col] = np.full(n, np.nan)
                else:
                    # 保证一维（有些字段是标量/一维；二维统计量这里不展开）
                    data[col] = np.asarray(arr).reshape(-1) if arr.ndim == 1 else np.asarray(arr)[:,0]

            df_beam = pd.DataFrame(data)

            if per_beam:
                # 每束单独输出
                out_csv_beam = h5_path.with_name(f"{h5_path.stem}_{g[1:]}.csv")
                df_beam.to_csv(out_csv_beam, index=False)
                outputs.append(out_csv_beam)
            else:
                rows_all.append(df_beam)

    if per_beam:
        # 已经分别写过文件
        print(f"✅ {h5_path.name}: 已按束导出 {len(outputs)} 个 CSV")
        for p in outputs:
            print("   -", p)
        return

    if rows_all:
        df_all = pd.concat(rows_all, ignore_index=True)
        df_all.to_csv(out_csv, index=False)
        print(f"✅ {h5_path.name}: 已导出合并 CSV -> {out_csv}（共 {len(df_all):,} 点）")
    else:
        print(f"⚠️ {h5_path.name}: 未找到可用束/经纬度，未生成 CSV。")

def batch_atl08_to_csv(file_list, out_dir=None, per_beam=False):
    """
    批量转换多个 ATL08 .h5 文件为 CSV。
    - out_dir: 输出目录（不传则和源文件同目录）
    - per_beam: 是否每束一个 CSV
    """
    for f in file_list:
        f = Path(f)
        if out_dir and not per_beam:
            # 合并 CSV 的输出改到 out_dir
            out_path = Path(out_dir) / f"{f.stem}.csv"
            Path(out_dir).mkdir(parents=True, exist_ok=True)
            atl08_h5_to_csv(f, out_csv=out_path, per_beam=False)
        else:
            # 同目录输出，或每束 CSV
            if out_dir and per_beam:
                Path(out_dir).mkdir(parents=True, exist_ok=True)
                # 临时在当前目录导出，再移动
                atl08_h5_to_csv(f, out_csv=None, per_beam=True)
                # per_beam 模式已在函数内部分束命名并保存于源目录；如需移动可额外写搬运逻辑
            else:
                atl08_h5_to_csv(f, out_csv=None, per_beam=per_beam)

In [46]:
atl08_h5_to_csv("/Users/a1-6/Self_program/underground_water/Jan_Nov_Dec/ATL08_007-20251005_180052/ATL08_20221113083320_08191702_007_01.h5")
# 输出：/path/to/ATL08_20220102113835_01711406_007_01.csv

✅ ATL08_20221113083320_08191702_007_01.h5: 已导出合并 CSV -> /Users/a1-6/Self_program/underground_water/Jan_Nov_Dec/ATL08_007-20251005_180052/ATL08_20221113083320_08191702_007_01.csv（共 130,124 点）
